In [1]:
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from DoN_radii2 import compute_DoN_feature_vector
import sys
import os
import colorsys
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
import numpy as np


In [88]:
# Load two point clouds with absolute paths
# pointcloud1 = o3d.io.read_point_cloud("/Users/noahbucher/Documents_local/Plant_reconstruction/ppheno/data/melonCycle/2024-08-01/A-4_2024-08-01/m_pc_A-4_2024-08-01_dense_03.ply")
# pointcloud2 = o3d.io.read_point_cloud("/Users/noahbucher/Documents_local/Plant_reconstruction/ppheno/data/melonCycle/2024-08-01/A-4_2024-08-01/pc_A-4_2024-08-01_dense_02_plant_cluster.ply")

# pointcloud2 = o3d.io.read_point_cloud("/Users/noahbucher/Documents_local/Plant_reconstruction/ppheno/data/melonCycle/2024-08-01/A-4_2024-08-01/pc_A-4_2024-08-01_dense_02_plant_only_cluster.ply")
# pointcloud2 = o3d.io.read_point_cloud("/Users/noahbucher/Documents_local/Plant_reconstruction/ppheno/data/melonCycle/2024-08-01/A-1_2024-08-01/m_pc_A-1_2024-08-01_dense_03.ply")

pointcloud2 = o3d.io.read_point_cloud("/Users/noahbucher/Documents_local/Plant_reconstruction/ppheno/data/melonCycle/2024-08-01/A-5_2024-08-01/m_pc_A-5_2024-08-01_dense_03.ply")
pointcloud1 = o3d.io.read_point_cloud("/Users/noahbucher/Documents_local/Plant_reconstruction/ppheno/data/melonCycle/2024-08-01/A-5_2024-08-01/pc_A-5_2024-08-01_dense_02_plant_cluster.ply")

In [89]:
# downsample the manual pointcloud

pointcloud1 = pointcloud1.voxel_down_sample(voxel_size=0.03)
cl, ind = pointcloud1.remove_statistical_outlier(nb_neighbors=20, std_ratio=2.0)
pcd = pointcloud1.select_by_index(ind)

In [90]:
def estimate_voxel_size_sklearn(pcd, sample_ratio=0.01, num_neighbors=20):
    """
    Estimate voxel size based on nearest neighbor distances using scikit-learn's NearestNeighbors.
    
    Args:
        pcd (o3d.geometry.PointCloud): The input point cloud.
        sample_ratio (float): Fraction of points to sample from the point cloud.
        num_neighbors (int): Number of neighbors for distance estimation.
    
    Returns:
        float: Estimated voxel size.
    """
    points = np.asarray(pcd.points)
    n_points = len(points)

    # Sample points if the point cloud is too large
    sample_size = max(1, int(n_points * sample_ratio))
    sample_points = points[np.random.choice(n_points, sample_size, replace=False)]

    # Use scikit-learn's NearestNeighbors for efficient KDTree-like search
    nbrs = NearestNeighbors(n_neighbors=num_neighbors, algorithm='auto').fit(sample_points)
    distances, _ = nbrs.kneighbors(sample_points)

    # Compute average distance to neighbors (ignore distance to self)
    avg_distances = np.mean(distances[:, 1:], axis=1)

    return np.mean(avg_distances)

In [91]:
# Estimate voxel size using scikit-learn's NearestNeighbors
estimated_voxel_size_sklearn = estimate_voxel_size_sklearn(pointcloud1, sample_ratio=0.9)
print(f"Estimated Voxel Size (scikit-learn): {estimated_voxel_size_sklearn}")

Estimated Voxel Size (scikit-learn): 0.05243471638194016


In [92]:
# Estimate voxel size using scikit-learn's NearestNeighbors
estimated_voxel_size_sklearn = estimate_voxel_size_sklearn(pointcloud2, sample_ratio=0.05)
print(f"Estimated Voxel Size (scikit-learn): {estimated_voxel_size_sklearn}")

Estimated Voxel Size (scikit-learn): 0.06355559792481405


In [93]:
voxel_size = 0.01  # Adjust this based on the resolution you need

voxel_size_1 = estimate_voxel_size_sklearn(pointcloud1, sample_ratio=0.5)
voxel_size_2 = estimate_voxel_size_sklearn(pointcloud2, sample_ratio=0.05)

voxel_grid1 = o3d.geometry.VoxelGrid.create_from_point_cloud(pointcloud1, voxel_size=voxel_size_2)
voxel_grid2 = o3d.geometry.VoxelGrid.create_from_point_cloud(pointcloud2, voxel_size=voxel_size_2)

In [94]:
def compute_voxel_iou(voxel_grid1, voxel_grid2):
    # Get the voxel coordinates for both grids
    voxels1 = set([tuple(voxel.grid_index) for voxel in voxel_grid1.get_voxels()])
    voxels2 = set([tuple(voxel.grid_index) for voxel in voxel_grid2.get_voxels()])

    # Compute intersection and union
    intersection = len(voxels1.intersection(voxels2))
    union = len(voxels1.union(voxels2))

    if union == 0:
        return 0.0

    return intersection / union

In [95]:
# Compute volumetric IoU
voxel_iou = compute_voxel_iou(voxel_grid1, voxel_grid2)
print(f"Voxel-based IoU: {voxel_iou}")

Voxel-based IoU: 0.6820039083318529


In [4]:
def feature_vector_DoN_Hue_N_coo(pcd, radius_small, radius_large):
    points = np.asarray(pcd.points)
    
    if not pcd.has_normals():
        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamKNN(knn=30))
    normals = np.asarray(pcd.normals)

    if pcd.has_colors():
        colors_rgb = np.asarray(pcd.colors)
        hues = np.array([colorsys.rgb_to_hsv(r, g, b)[0] for r, g, b in colors_rgb])
    else:
        hues = np.zeros(len(points))

    DoN_magnitudes = compute_DoN_feature_vector(pcd, radius_small, radius_large)
    
    feature_vector = np.hstack((points, normals, hues[:, np.newaxis], DoN_magnitudes[:, np.newaxis]))
    return feature_vector


In [5]:
def compute_iou(predicted_labels, ground_truth_labels):
    intersection = np.sum(np.logical_and(predicted_labels, ground_truth_labels))
    union = np.sum(np.logical_or(predicted_labels, ground_truth_labels))
    
    if union == 0:
        return 0.0
    
    return intersection / union


In [6]:
import open3d as o3d
import numpy as np

def voxelize_pointcloud(pcd, voxel_size):
    voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(pcd, voxel_size=voxel_size)
    return voxel_grid

def compute_voxel_iou(voxel_grid1, voxel_grid2):
    # Get the voxel coordinates for both grids
    voxels1 = set([tuple(voxel.grid_index) for voxel in voxel_grid1.get_voxels()])
    voxels2 = set([tuple(voxel.grid_index) for voxel in voxel_grid2.get_voxels()])

    # Compute intersection and union
    intersection = len(voxels1.intersection(voxels2))
    union = len(voxels1.union(voxels2))

    if union == 0:
        return 0.0

    return intersection / union

# Load two point clouds
pcd1 = o3d.io.read_point_cloud("/path_to_pointcloud1.ply")
pcd2 = o3d.io.read_point_cloud("/path_to_pointcloud2.ply")

# Voxelize the point clouds
voxel_size = 0.01  # Adjust this based on the resolution you need
voxel_grid1 = voxelize_pointcloud(pcd1, voxel_size)
voxel_grid2 = voxelize_pointcloud(pcd2, voxel_size)

# Compute volumetric IoU
voxel_iou = compute_voxel_iou(voxel_grid1, voxel_grid2)
print(f"Voxel-based IoU: {voxel_iou}")


[Open3D WARNING] Read PLY failed: unable to open file: /path_to_pointcloud1.ply
[Open3D WARNING] Read PLY failed: unable to open file: /path_to_pointcloud2.ply
Voxel-based IoU: 0.0


RPly: Unable to open file
RPly: Unable to open file


: 

In [4]:
def objective_function(pcd, ground_truth_pcd, radius_small, radius_large, weights, n_clusters=5):
    weighted_features = extract_and_weight_features(pcd, radius_small, radius_large, weights)
    
    gmm = GaussianMixture(n_components=n_clusters, random_state=42).fit(weighted_features)
    labels = gmm.predict(weighted_features)
    
    plant_cluster = identify_plant_cluster(pcd, labels)
    
    predicted_plant_mask = labels == plant_cluster
    ground_truth_points = np.asarray(ground_truth_pcd.points)
    full_pcd_points = np.asarray(pcd.points)
    
    ground_truth_mask = np.isin(full_pcd_points, ground_truth_points)
    
    iou_score = compute_iou(predicted_plant_mask, ground_truth_mask)
    
    return iou_score


In [5]:
def identify_plant_cluster(pcd, labels):
    colors = np.asarray(pcd.colors)
    hue_values = np.array([colorsys.rgb_to_hsv(r, g, b)[0] * 360 for r, g, b in colors])
    
    green_hue_min, green_hue_max = 80, 100
    
    green_counts = []
    n_clusters = len(np.unique(labels))  # Count the number of unique clusters
    for i in range(n_clusters):
        cluster_indices = (labels == i)
        green_count = np.sum((cluster_indices) & (green_hue_min <= hue_values) & (hue_values <= green_hue_max))
        green_counts.append(green_count)
    
    plant_cluster = np.argmax(green_counts)
    return plant_cluster


In [7]:
# Load the point cloud
relative_pcd_path = "/Users/noahbucher/Documents_local/Plant_reconstruction/ppheno/data/melonCycle/2024-08-01/A-1_2024-08-01/pc_A-1_2024-08-01_dense_02.ply"
pcd = o3d.io.read_point_cloud(relative_pcd_path)

# Define parameters
radius_small = 0.4
radius_large = 2.0
n_clusters = 5
weights = {
    'spatial': 0.3,
    'normals': 0.0,
    'hue': 1.0,
    'don': 0.0
}

# Load the ground truth point cloud
ground_truth_pcd_path = "/Users/noahbucher/Documents_local/Plant_reconstruction/ppheno/data/melonCycle/2024-08-01/A-1_2024-08-01/m_pc_A-1_2024-08-01_dense_03.ply"  # Adjust the path
ground_truth_pcd = o3d.io.read_point_cloud(ground_truth_pcd_path)

# Compute IoU score using the objective function
iou_score = objective_function(pcd, ground_truth_pcd, radius_small, radius_large, weights, n_clusters)
print(f"Computed IoU score: {iou_score}")


NameError: name 'extract_and_weight_features' is not defined